In [1]:
print("Starting stuff")
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from secrets import *
import time
import random
import numpy
import pandas
import os
# print(secrets.passkey)
import json
import urllib.request

Starting stuff


In [2]:
driver = webdriver.Firefox()
time.sleep(2)
tag='artistsofinstagram'
driver.get('https://www.instagram.com/explore/tags/'+tag+'/')
time.sleep(2)

In [3]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [23]:
posts=[]
with open('database.json','r') as db:
    data=db.read()
# print(data[:-1])
print(data[2808:2814])
posts=json.loads(data[10:-1])
print(posts[0])

</a></


JSONDecodeError: Extra data: line 1 column 2812 (char 2811)

In [5]:

leads = soup.find_all("a")
print(len(leads))

46


## Find post links from hashtag page

In [6]:
for lead in leads:
    img=lead.find("img", class_="FFVAD")
    if(img):
        post={}
        post["url"]=lead.attrs['href']
        post["alt"]=img['alt']
        post["raw_link"]=img['srcset']
        post["tag"]=tag
        posts.append(post)
        
for post in posts:
    part=post["raw_link"].split(",")
    clean=part[-1].split()
    post["src"]=clean[0]

## Find details of post

In [7]:
for post in posts:
    url='https://www.instagram.com'+post["url"]
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find("img", class_="FFVAD")
    try:
        post["raw_link"]=img['srcset']
        
        part=post["raw_link"].split(",")
        clean=part[-1].split()
        post["src"]=clean[0]
            
        post["size"]=clean[1]
         
    except:
        print("Some error")
    
    user = soup.find("h2").div.a
    post["user"]=user["href"]
    post["username"]=user.string
    
    raw_description=soup.find("h2").next_sibling
    post["plaintext_description"]=raw_description.get_text()
    post["rich_description"]=str(raw_description)
#     print(post["descriptionplain"], "\n \n")
#     print(post["rich_description"])
    
    

Some error


## Save images to /img/tag

In [8]:
directory= os.path.join(os.path.expanduser('~'),'Insta-hashtag', 'img', tag)
if not os.path.isdir(directory):
    os.makedirs(directory) 
    
for post in posts: 
    pic_url=post["src"]
    name=post["url"][3:-1]+'.jpg'
    path = os.path.join(directory,name)
    if (not os.path.isfile(path)):
        try:
            urllib.request.urlretrieve(pic_url, path)
            print("...",name )
        except:
            print("some error")
print("Completed")

... B_xw82ol63w.jpg
... B_xIwDnAWbK.jpg
... B_wI7B4l7pJ.jpg
... B_y-yJnlvhp.jpg
... B_y-x3FJCv5.jpg
... B_y-xFQHBO9.jpg
... B_y-xApF4-3.jpg
... B_y-oSaJRbw.jpg
... B_y-tfZjHOR.jpg
... B_y-skUloJ2.jpg
... B_y-rydBJm8.jpg
... B_y-r3rFlHY.jpg
... B_y-q9oDslP.jpg
... B_y-quYKalm.jpg
... B_y-qESFLgJ.jpg
Completed


In [13]:
with open('database.json','a') as db:
    db.write("var data=")
    db.write(json.dumps(posts))
    db.write(";")